# SYDE 556/750 --- Assignment 4
**Student ID: 20814292**

*Note:* Please include your numerical student ID only, do *not* include your name.

*Note:* Refer to the [PDF](https://github.com/celiasmith/syde556-f22/raw/master/assignments/assignment_04/syde556_assignment_04.pdf) for the full instructions (including some hints), this notebook contains abbreviated instructions only. Cells you need to fill out are marked with a "writing hand" symbol. Of course, you can add new cells in between the instructions, but please leave the instructions intact to facilitate marking.

In [58]:
# Import numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt

# Import nengo and some helper functions for Q1
import nengo
from nengo.utils.ensemble import tuning_curves
from nengo.utils.connection import eval_point_decoding

# Fix the numpy random seed for reproducible results
np.random.seed(18945)

# Some formating options
%config InlineBackend.figure_formats = ['svg']

# 1. Building an ensemble of neurons

In [59]:
tau_RC = 20 / 1000
tau_ref = 2 / 1000
noise = 0.1 * 200

**a) Tuning curves.** Plot the population tuning curves. Plot the representation accuracy plot ($x - \hat{x}$). Compute and report the RMSE.

In [60]:
model = nengo.Network(label="Two Neurons")
with model:
    ensemble = nengo.Ensemble(
        100,
        dimensions=1,
        intercepts=nengo.dists.Uniform(-1.0, 1.0),
        max_rates=nengo.dists.Uniform(100, 200),  
        neuron_type=nengo.LIF(tau_rc=tau_RC, tau_ref=tau_ref),
        noise=noise
    )

    # Dummy identity connection for decoder computation
    conn = nengo.Connection(ensemble, ensemble, synapse=0)

# Build and simulate the model to get tuning curves and decoders
with nengo.Simulator(model) as sim:
    # Compute decoders under noise
    eval_points, activities = nengo.utils.ensemble.tuning_curves(ensemble, sim)

    eval_points_decoding = np.linspace(-1, 1, 500).reshape(-1, 1)  # Uniform test points
    _, _, decoders = eval_point_decoding(
        conn, sim, eval_points_decoding,
    )
    decoded_output = eval_points_decoding @ decoders.T
    
# Compute representation error and RMSE
true_values = eval_points_decoding.flatten()
decoded_values = decoded_output.mean(axis=1)
representation_error = true_values - decoded_values
rmse = np.sqrt(np.mean(representation_error**2))

# Plot tuning curves
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Tuning Curves")
plt.xlabel("Input ($x$)")
plt.ylabel("Neural Activity (Hz)")
plt.plot(eval_points, activities)
plt.grid()

# Plot representation accuracy
plt.subplot(1, 2, 2)
plt.title("Representation Accuracy")
plt.xlabel("True Input ($x$)")
plt.ylabel("Representation Error ($x - \hat{x}$)")
plt.plot(true_values, representation_error, label=f"RMSE = {rmse:.4f}")
plt.axhline(0, color='black', linestyle='--', linewidth=0.8)
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

# Print RMSE
print(f"Root Mean Square Error (RMSE): {rmse:.4f}")

ValidationError: Ensemble.noise: Must be of type 'Process' (got type 'float').

**b) RMSE and radius.** Compute the RMSE for (at least) the four different radii $0.5$, $1$, $2$, and $4$. Plot your results. Compute the RMSE for (at least) the four different radii $0.5$, $1$, $2$, and $4$. Plot your results. Make sure your neurons have the same (relative, i.e., scaled by the radius) $x$-intercepts and maximum rates across all experiments.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**c) Discussion.** What mathematical relationship between the radius and the RMSE do you observe (write down an equation)? Explain why this is the case.

✍ \<YOUR SOLUTION HERE\>

**d) RMSE and refractory period.** What happens to the RMSE and the tuning curves as $\tau_\mathrm{ref}$ changes between $1$ and $5\,\mathrm{ms}$? Plot the tuning curves for at least four different $\tau_\mathrm{ref}$ and produce a plot showing the RMSE over $\tau_\mathrm{ref}$. Again, make sure to use the same neuron ensemble parameters in all your trials.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**e) RMSE and membrane time constant.** What happens to the RMSE and the tuning curves as $\tau_\mathrm{RC}$ changes between $10$ and $100\,\mathrm{ms}$? Plot the tuning curves for at least four different $\tau_\mathrm{RC}$ and produce a plot showing the RMSE over $\tau_\mathrm{RC}$.  Again, make sure to use the same neuron ensemble parameters in all your trials.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**f) Discussion.** Discuss the last two results. Describe what happens to the tuning curves as $\tau_\mathrm{ref}$ and $\tau_\mathrm{RC}$ change (you do not need to come up with a mathematical relationship here). Explain why the change in tuning curve shape influences the RMSE in the way you observe.

✍ \<YOUR SOLUTION HERE\>

# 2. Connecting neurons

**a) Computing the identity function.** Show the input value and the decoded values from the two  ensembles in three separate plots. Run the simulation for $0.5\,\mathrm{s}$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**b) Computing an affine transformation.** Make a new version of the model where instead of computing the identity function, it computes $y(t) = 1 - 2x(t)$. Show the same graphs as in part (a).

In [ ]:
# ✍ <YOUR SOLUTION HERE>

# 3. Dynamics

**a) Transforming the dynamical system.** Rewrite the linear dynamical system describing the integrator in terms of $\frac{\mathrm{d}\vec x(t)}{\mathrm{d}t} = \mathbf{A} \mathbf{x} + \mathbf{B} \mathbf{u}$, i.e., write down the matrices $\mathbf{A}$ and $\mathbf{B}$ (you can just use the equations from class, you do not have to re-derive the equations) What are the matrices $\mathbf{A}'$ and $\mathbf{B}'$ we have to use when implementing this system using the recurrent connection post-synaptic filter?

✍ \<YOUR SOLUTION HERE\>

**b) Integrator using spiking neurons.**  Show the input, the ideal integral, and the value represented by the ensemble when the input is a value of $0.9$ from $t=0.04$ to $t=1.0$ (and $0$ for other times). Run the simulation for $1.5\,\mathrm{s}$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**c) Discussion.** What is the expected ideal result, i.e., if we just mathematically computed the integral of the input, what is the equation describing the integral? How does the simulated output compare to that ideal?

✍ \<YOUR SOLUTION HERE\>

**d) Simulation using rate neurons.** Change the neural simulation to rate mode. Re-run the simulation in rate mode. Show the resulting plots.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**e) Discussion.** How does this compare to the result in part (b)? What deviations from the ideal do you still observe? Where do those deviations come from?

✍ \<YOUR SOLUTION HERE\>

**f) Integration of a shorter input pulse.** Returning to spiking mode, change the input to be a value of $0.9$ from $t=0.04$ to $0.16$. Show the same plots as before (the input, the ideal, and the value represented by the ensemble over $1.5\,\mathrm{s}$).

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**g) Discussion.** How does this compare to (b)? What is the ideal equation? Does it work as intended? If not, why is it better or worse?

✍ \<YOUR SOLUTION HERE\>

**h) Input ramp.** Change the input to a ramp input from $0$ to $0.9$ from $t=0$ to $t=0.45$ (and $0$ for $t>0.45$). Show the same plots as in the previous parts of this question.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**i) Discussion.** What does the ensemble end up representing, and why? What is the (ideal) equation for the curve traced out by the ensemble?

✍ \<YOUR SOLUTION HERE\>

**j) Sinusoidal input.** Change the input to $5\sin(5t)$. Show the same plots as before.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**k) Discussion.** What should the value represented by the ensemble be? Write the equation. How well does it do? What are the differences between the model's behaviour and the expected ideal behaviour and why do these differences occur?

✍ \<YOUR SOLUTION HERE\>

**l) Controlled integrator.** Implement a controlled integrator over two popoulations. Demonstrate that it's working as expected.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**m) 🌟 Bonus question.** Implement a nonlinear dynamical system we have not seen in class (and that is not in the book). Demonstrate that it's working as expected

✍ \<YOUR SOLUTION HERE\>

In [ ]:
# ✍ <YOUR SOLUTION HERE>